In [ ]:
from vizdoom import *
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, MultiBinary
import cv2
from stable_baselines3.common import env_checker
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Test

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import time
import os
import cv2
import numpy as np
from deadlyCorridor import VizDoomGym, make_env

def test_model(model_path, config_file, n_episodes=1, render=True, record_video=True):
    # Crear entorno
    env = DummyVecEnv([make_env(config_file, render=render)])

    # Cargar modelo
    model = PPO.load(model_path)

    for ep in range(n_episodes):
        obs = env.reset()
        done = False
        score = 0
        print(f"\n--- Episodio {ep + 1} ---")

        # Video config
        video_writer = None
        if record_video:
            width, height = 320, 240  # tamaño real del screen_buffer
            video_path = f"video_ep{ep + 1}.avi"
            fps = 35
            video_writer = cv2.VideoWriter(
                video_path,
                cv2.VideoWriter_fourcc(*'XVID'),
                fps,
                (width, height)
            )

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _ = env.step(action)
            score += reward[0]

            # Obtener frame a color del screen_buffer
            if record_video:
                state = env.get_attr("game")[0].get_state()
                if state is not None:
                    frame = state.screen_buffer  # (3, H, W)
                    frame = np.transpose(frame, (1, 2, 0))  # → (H, W, C)
                    frame = cv2.resize(frame, (width, height))
                    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                    video_writer.write(frame)

            if render:
                time.sleep(0.06)

        print(f"🎯 Score episodio {ep + 1}: {score:.2f}")

        if record_video:
            video_writer.release()
            print(f"💾 Video guardado como: {video_path}")

    env.close()

if __name__ == "__main__":
    test_model(
        model_path="./models/deadly_corridor/stage_3/best_model.zip",
        config_file="deadly_corridor_s5_killcount.cfg",
        n_episodes=5,
        render=True,
        record_video=True
    )
